In [2]:
!pip install -U pip setuptools wheel --user
!pip install -U spacy

  Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
  Using cached setuptools-51.1.0.post20201221-py3-none-any.whl (2.0 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import numpy as np
import pandas as pd
import spaCy

import requests
import bs4

In [ ]:
response = reqquest.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q=election&api-key=yourkey")